In [1]:
import pandas as pd
import numpy as np
from soundfile import read as sf_read
from librosa import load as lib_load
import tensorflow_io as tfio
import tensorflow as tf

In [2]:
tracks = pd.read_csv('./data/processed_genres.csv')
tracks = np.array(tracks.loc[:, ['parent_genre_id', 'fpath']])

dim = 44100 * 3

In [3]:
# Pure tensorflow test
for x in range(5000):
    fpath = './data/fma_medium' + tracks[x][-1]
    audio = tfio.audio.AudioIOTensor(fpath)
    shape = tf.cast(audio.shape, tf.dtypes.int32)
    start_idx = tf.random.uniform(shape=[], minval=0, maxval=shape[0] - dim, dtype=tf.dtypes.int32)
    audio_slice = audio[start_idx:start_idx + dim]
    # Convert to one channel
    # either by averaging stereo channels or removing extra dim on mono
    if tf.equal(shape[-1], 2):
        audio_tensor = tf.reduce_mean(audio_slice, axis=[-1])
    else:
        audio_tensor = tf.squeeze(audio_slice, axis=[-1])

InvalidArgumentError: {{function_node __wrapped__IO>AudioReadableRead_device_/job:localhost/replica:0/task:0/device:CPU:0}} read 264600 from 1190107 failed: 0 [Op:IO>AudioReadableRead]

In [4]:
# Librosa test
for x in range(5000):
    fpath = './data/fma_medium' + tracks[x][-1]
    audio = tfio.audio.AudioIOTensor(fpath)
    shape = tf.cast(audio.shape, tf.dtypes.int32)
    start_idx = tf.random.uniform(shape=[], minval=0, maxval=tf.cast(tf.divide(shape[0] - dim, 44100), tf.dtypes.float32), dtype=tf.dtypes.float32)
    audio_slice, _ = lib_load(fpath, sr=44100, mono=True, offset=start_idx, duration=3.0)

In [5]:
# Soundfile test
for x in range(5000):
    fpath = './data/fma_medium' + tracks[x][-1]
    audio = tfio.audio.AudioIOTensor(fpath)
    shape = tf.cast(audio.shape, tf.dtypes.int32)
    start_idx = tf.random.uniform(shape=[], minval=0, maxval=shape[0] - dim, dtype=tf.dtypes.int32)
    audio_slice, _ = sf_read(fpath, frames=dim, start=start_idx)
    # Convert to one channel by averaging stereo channels
    if tf.equal(shape[-1], 2):
        audio_tensor = tf.reduce_mean(audio_slice, axis=[-1])